In [1]:
import pandas as pd  # Import pandas and use shorthand notation pd to save on typing
from pandas import Series, DataFrame 
import numpy as np                            
import statsmodels.api as sm 
import scipy as sp
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [2]:
electricityPrices = pd.read_csv(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\electricityprices.csv")
electricityPrices['Year'] = electricityPrices['Month beginning'].str[-4:]
del electricityPrices['Month beginning']
electricityPrices = DataFrame({'Average Electricity Price': electricityPrices.groupby('Year')['Price'].mean().round(3)})
electricityPrices.index = electricityPrices.index.astype(int)
electricityPrices

FileNotFoundError: File b'C:\\Users\\Kiran\\Documents\\University\\Economics Yr 3\\My Courses\\EC3310 Coding\\GithubRepo\\electricityprices.csv' does not exist

In [ ]:
busFares = pd.read_excel(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\busfares.xlsx")
busFares = busFares.drop(range(13))
busFares = busFares[['Year','Great Britain']]
busFares = busFares.set_index('Year')
busFares.columns = ['Bus Fare Cost']
busFares

In [ ]:
total = pd.concat([electricityPrices, busFares], axis=1)
total

In [ ]:
charger_data = {'Slow':[1325, 1611, 2126, 2448, 2228, 2451, 2743, 3293, 5947], 
        'Fast': [178,1193,3309,4734,5946,7143,8754,11459,16457], 
        'Rapid':[34,79,284,844,1710,2153,2663,4025,6212],
        'Ultra-rapid':[None,None,None,None,None,None,None,341,876]}
df = pd.DataFrame(charger_data, index=[2011,2012,2013,2014,2015,2016,2017,2018,2019])
df["Sum"] = df.sum(axis=1).astype(int)
df

In [ ]:
total['Charging Connectors'] = df['Sum']
total['ULEZ Variable'] = 1
range_data = [None, 244, 84, 73, 118, 79, 81, 114, 128, 177, 190, 220, 256]
total['Range of Electric Vehicle (miles)'] = range_data

In [ ]:
electricityAvailable = pd.read_excel(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\electricityavailable.xlsx")
electricityAvailable = electricityAvailable.set_index("Year") 
total = pd.concat([total, electricityAvailable], axis=1)
total = total.rename({'Electricity Available': 'Electricity Available (TWh)'}, axis=1).round(3)
total

In [ ]:
import requests
import json

api = requests.get('https://api.ons.gov.uk/dataset/MM23/timeseries/CJXN/data')
 
# print (api.json())
newcars_dict = json.loads(api.content)
years = newcars_dict['years']
filtered = []
for year in years:
    if int(year['year']) > 2007:
        filtered.append(int(float(year['value'])))
newCars = np.asarray(filtered)
total['Price of New Cars'] = newCars
total

In [ ]:
from bs4 import BeautifulSoup
import re

url = ('https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/k8a5/bb')
page = requests.get(url).text
soup = BeautifulSoup(page, "html.parser")
rows = soup.find_all("table")[1].find('tbody').find_all('tr')

co_data = []
for row in rows:
    cells = row.find_all("td")
    years = int(cells[0].text)
    data = int(float(cells[1].text))
    co_data.append([years, data])

co_df = pd.DataFrame(co_data, columns=['Year', 'Carbon Monoxide (thousand tonnes)'])
co_df = co_df.drop(range(18))
co_df = co_df.set_index('Year')
total = pd.concat([total, co_df], axis=1)
total

In [ ]:
total.to_csv("kiran_csv")